In [1]:
import requests
import json
import csv
import pandas as pd
from matplotlib import pyplot as plt
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC #using alias as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import os
from PIL import Image
import glob

In [2]:
# IMPORTANT

# need new user to set up to view floor plan images:

# 1. get current work directory:

directory = os.getcwd()

directory

# 2. add "Floor Plans" folder to directory

    
# https://www.geeksforgeeks.org/get-directory-of-current-python-script/
# https://www.geeksforgeeks.org/python-pil-image-open-method/

'/Users/clt-user/Downloads'

In [3]:
# webscraping function

def show_rooms():
    
    
    year_to_book = input("Please enter the current year: ")
    month_to_book = input("Please enter the month in which you'd like to search (Jan-Feb-Mar-Apr-May-Jun-Jul-Aug-Sep-Oct-Nov-Dec): ")
    
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    
    if month_to_book not in months:
        
        print("Invalid month, please try again.")
        main()
        
    else:
        
        pass
    
    
    date_to_book = input("Please enter the day you'd like to search as number: ")
    
    
    
    global options
    
    options = Options()
    options.add_argument("start-maximized")

    global PATH
    
    PATH = Service("/Users/clt-user/Desktop/selenium/chromedriver.exe")
    
    
    driver = webdriver.Chrome(service=PATH, options=options)

    # prevents previous data from being used in new trial
    
    if os.path.exists("library.csv"):
        os.remove("library.csv")
    else:
        pass
    
    
    library_slots = open("library.csv", "w", newline="", encoding="utf-8") #creating a csv file to write rows
    csv_writer = csv.writer(library_slots)
    csv_writer.writerow(["Time", "Day", "Date", "Room", "Availability"])


    driver.get("https://www.binghamton.edu/libraries/services/reservations/index.html")

    
    try:
        #-------------------------Bartle Library----------------------------
        #Click Bartle library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN BARTLE"))
        )

        element.click()

        #Click to open drop down menu for bartle library
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']"))
        )

        element.click()


        #Click Show All in the drop down menu
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']/option[5]"))
        )

        element.click()


        #Click Go To Date to open Calendar
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
        )

        element.click()

        #Click datepicker to select month
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
        )

        element.click()

        #click datepicker to select year
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
        )

        element.click()

        #selecting year given by user in calendar
        element=   WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
        element.click()

        #Selecting month given by user in calendar
        months = driver.find_elements(By.TAG_NAME, "span")
        for month in months:
            if month.text == month_to_book:
                month.click()
                time.sleep(5)
                break

        #selecting date given by user in calender (number date)
        dates = driver.find_elements(By.TAG_NAME, "td")
        
        flag = False
        for date in dates:
            if date.text == "1":
                flag = True

            if flag == False:
                continue

            if date.text == date_to_book:
                date.click()
                time.sleep(5)
                break

        flag = False
        time.sleep(10)
        
        tday = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if tday[0] == "0":
            tday = tday[1:]
        
        #scraping available slots for the user given date
        if tday == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")
        
        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]
            if "Media Viewing Room" not in details:
                room_ = data[5]
                available_ = data[6]            
            else:
                room_ = data[5] + " " + data[6] + " " + data[7] + " " + data[8]
                available_ = data[9]

            csv_writer.writerow([time_, day_, date_, room_, available_])

            #new_row = {"Time": time_, "Day": day_, "Date": date_, "Room": room_, "Availability": available_}
            #schedule.append(new_row, ignore_index = True)

        driver.back()
        driver.back()

        #-------------------------Science Library----------------------------

        #Click Science library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN THE SCIENCE LIBRARY"))
        )

        element.click()

        #Click Go To Date to open Calendar
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
        )

        element.click()

        #Click datepicker to select month
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
        )

        element.click()

        #click datepicker to select year
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
        )

        element.click()

        #selecting year given by user in calendar
        element=   WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
        element.click()

        #Selecting month given by user in calendar
        months = driver.find_elements(By.TAG_NAME, "span")
        for month in months:
            if month.text == month_to_book:
                month.click()
                time.sleep(5)
                break

        #selecting date given by user in calender (number date)
        dates = driver.find_elements(By.TAG_NAME, "td")
        
        flag = False
        for date in dates:
            if date.text == "1":
                flag = True

            if flag == False:
                continue

            if date.text == date_to_book:
                date.click()
                time.sleep(5)
                break

        flag = False
        time.sleep(10)

        #time.sleep(10)

        tday = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if tday[0] == "0":
            tday = tday[1:]
        
        #scraping available slots for the user given date
        if tday == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")
 
        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]
            room_ = data[5] + " " + data[6] + " " + data[7]
            available_ = data[8]

            csv_writer.writerow([time_, day_, date_, room_, available_])

            #new_row = {"Time": time_, "Day": day_, "Date": date_, "Room": room_, "Availability": available_}
            #schedule.append(new_row, ignore_index = True)


        driver.back()

        #-------------------------UDC Library----------------------------

        #Click UDC library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN THE UDC"))
        )

        element.click()

        #Click Go To Date to open Calendar
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
        )

        element.click()

        #Click datepicker to select month
        element = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
        )

        element.click()

        #click datepicker to select year
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
        )

        element.click()

        #selecting year given by user in calendar
        element=   WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
        element.click()

        #Selecting month given by user in calendar
        months = driver.find_elements(By.TAG_NAME, "span")
        for month in months:
            if month.text == month_to_book:
                month.click()
                time.sleep(5)
                break

        #selecting date given by user in calender (number date)
        dates = driver.find_elements(By.TAG_NAME, "td")
        
        flag = False
        for date in dates:
            if date.text == "1":
                flag = True

            if flag == False:
                continue

            if date.text == date_to_book:
                date.click()
                time.sleep(5)
                break

        flag = False
        time.sleep(10)
        
        tday = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if tday[0] == "0":
            tday = tday[1:]
        
        #scraping available slots for the user given date
        if tday == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

            
        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]
            room_ = data[5] + " " + data[6] + " " + data[7]
            available_ = data[8]

            csv_writer.writerow([time_, day_, date_, room_, available_])

            #new_row = {"Time": time_, "Day": day_, "Date": date_, "Room": room_, "Availability": available_}
            #schedule.append(new_row, ignore_index = True)

        library_slots.close()

    finally:
        driver.quit()
    


        
    global library_csv
    
    library_csv = pd.read_csv("library.csv") #read csv file which was recently written
    
    print(library_csv)
    
    


In [4]:
# pre-filter function: gives user option to filter or proceed with all rooms that were scraped
    
def filter_rooms():
            
    further = input("Would you like to specify further? (y/n)")

    if further == "y":

        filter_slots()

    else:

        print("Would you like to book one of these rooms, start a new search, or exit?")
        print("1. Book Room")
        print("2. New Search")
        print("3. Exit")
        next_step = int(input("Selection (1-3): "))

        if next_step == 1:

            room_list = library_csv
            room_num = int(input("Please enter the corresponding option number of the room: "))
            book_this_room = room_list.loc[room_num]
            print(book_this_room)
            book_room(book_this_room)

        elif next_step == 2:

            show_rooms()

        elif next_step == 3:
        
            print("Happy studying!")
            
            

In [5]:
# filtering function: can filter results from above based on time or location

# one can be done after the other, or the user can choose not to do one or both
    
def filter_slots():

    global library_filter
    
    choice = input("Would you like to specify time or location? ")

    # filter by time first:
    
    
    
    if choice == "time":

        valid = False
        
        while valid == False:
        
            library_filter = library_csv.copy()
            
            time_choice = input("Please enter a starting time (H:MMam/pm): ")

            library_filter.drop(library_filter[library_filter["Time"] != time_choice].index, inplace = True)

            library_filter = library_filter.reset_index(drop = True) # new line

            if len(library_filter) == 0:

                print("No rooms are available at this time, please search again.")

            else:

                print(library_filter)

                valid = True

        further_filter = input("Would you like to now specify by location? (y/n) ")

        valid = False
        
        if further_filter == "y":

            while valid == False:  
                
                library_filter_2 = library_filter.copy()
                
                print("Studying locations are as follows:")
                print("1. Bartle Library")
                print("2. Science Library")
                print("3. University Downtown Center")
                location_choice = int(input("Where would you like to study? (1-3)"))

                if location_choice == 1:

                    ln = library_filter_2[library_filter_2["Room"].str.contains("LN")]

                    ls = library_filter_2[library_filter_2["Room"].str.contains("LS")]

                    bloomberg = library_filter_2[library_filter_2["Room"].str.contains("Bloomberg")]

                    room_list = pd.concat([ln, ls, bloomberg], ignore_index = True)

                    if len(room_list) == 0:
                        
                        print("No rooms are available at this location, please search again.")
                    
                    else:
                    
                        print(room_list)
                    
                        valid = True



                elif location_choice == 2:

                    blue = library_filter_2[library_filter_2["Room"].str.contains("Blue")]

                    grape = library_filter_2[library_filter_2["Room"].str.contains("Grape")]

                    grey = library_filter_2[library_filter_2["Room"].str.contains("Grey")]

                    orange = library_filter_2[library_filter_2["Room"].str.contains("Orange")]

                    room_list = pd.concat([blue, grape, grey, orange], ignore_index = True)

                    if len(room_list) == 0:
                        
                        print("No rooms are available at this location, please search again.")
                    
                    else:
                    
                        print(room_list)
                    
                        valid = True


                elif location_choice == 3:

                    room_list = library_filter_2[library_filter_2["Room"].str.contains("UDC")]

                    if len(room_list) == 0:
                        
                        print("No rooms are available at this location, please search again.")
                    
                    else:
                    
                        print(room_list)
                    
                        valid = True
                    

        else:   
            pass
        
        print("Would you like to book one of these rooms, start a new search, or exit?")
        print("1. Book Room")
        print("2. Change Filters")
        print("3. Search New Date")
        print("4. Exit")
                
        search_or_book = int(input("Selection (1-4): "))
                
                
        if search_or_book == 1:
                    
            room_list = library_filter
            room_num = int(input("Please enter the corresponding option number of the room: "))
            book_this_room = room_list.loc[room_num]
            print(book_this_room)
            book_room(book_this_room)
                    
                    # queue booking function
                    
        elif search_or_book == 2:
                    
            filter_slots()
                    # restart filter function
                    
        elif search_or_book == 3:
            
            main()
        
        # restart main function
        
        elif search_or_book == 4:
                    
            print("Goodbye, study hard!")
                    
    
    # filter by location first:                
                    
    elif choice == "location":
        
        valid = False
        
        while valid == False:
            
            library_filter = library_csv.copy()
        
            print("Studying locations are as follows:")
            print("1. Bartle Library")
            print("2. Science Library")
            print("3. University Downtown Center")
            location_choice = int(input("Where would you like to study? (1-3)"))

            global room_list2
            
            if location_choice == 1:

                ln = library_filter[library_filter["Room"].str.contains("LN")]

                ls = library_filter[library_filter["Room"].str.contains("LS")]

                bloomberg = library_filter[library_filter["Room"].str.contains("Bloomberg")]

                room_list = pd.concat([ln, ls, bloomberg], ignore_index = True)

                if len(room_list) == 0:

                    print("No rooms are available at this location, please search again.")

                else:

                    print(room_list)

                    #global room_list2
                    
                    room_list2 = room_list.copy()
                    
                    valid = True



            elif location_choice == 2:

                blue = library_filter[library_filter["Room"].str.contains("Blue")]

                grape = library_filter[library_filter["Room"].str.contains("Grape")]

                grey = library_filter[library_filter["Room"].str.contains("Grey")]

                orange = library_filter[library_filter["Room"].str.contains("Orange")]

                room_list = pd.concat([blue, grape, grey, orange], ignore_index = True)

                if len(room_list) == 0:

                    print("No rooms are available at this location, please search again.")

                else:

                    print(room_list)

                    #global room_list2
                    
                    room_list2 = room_list.copy()
                    
                    valid = True



            elif location_choice == 3:

                room_list = library_filter[library_filter["Room"].str.contains("UDC")]

                room_list = room_list.reset_index(drop = True)

                if len(room_list) == 0:

                    print("No rooms are available at this location, please search again.")

                else:

                    print(room_list)
                            
                    #global room_list2
                    
                    room_list2 = room_list.copy()

                    valid = True


        filter_further = input("Would you like to now filter by time? (y/n) ")
        
        
        if filter_further == "y":
            
            valid = False
                    
            #room_list2 = room_list.copy()
            
            while valid == False:
            
                room_list = room_list2.copy()
                
                time_choice = input("Please enter a starting time (H:MMam/pm): ")

                room_list.drop(room_list[room_list["Time"] != time_choice].index, inplace = True)

                room_list = room_list.reset_index(drop = True)
            
                if len(room_list) == 0:

                    print("No rooms are available at this location, please search again.")

                else:

                    print(room_list)

                    valid = True
            
        else:
            pass
            
        print("Would you like to book one of these rooms, start a new search, or exit?")
        print("1. Book Room")
        print("2. Change Filters")
        print("3. Search New Date")
        print("4. Exit")
                
        search_or_book = int(input("Selection (1-3): "))
                
                
        if search_or_book == 1:
                    
            room_num = int(input("Please enter the corresponding option number of the room: "))
            book_this_room = room_list.loc[room_num]
            print(book_this_room)
            book_room(book_this_room)
            
            # queue booking function
                    
        elif search_or_book == 2:
                    
            filter_slots()
                    # restart filter function
                    
        
        elif search_or_book == 3:
            
            main()
            
            # restart main function
        
        
        elif search_or_book == 4:
            
            print("Goodbye, study hard!")
            
            

In [6]:
# pre-booking function: calls 1 of 3 different functions that will book a certain room


def book_room(book_this_room):

    global month_to_book
    
    month_to_book = book_this_room["Date"][0:3]

    n = len(book_this_room["Date"])

    global year_to_book
    
    year_to_book = book_this_room["Date"][n-4:n]

    global date_to_book
    
    date_to_book = book_this_room["Date"].replace(year_to_book, "")

    global time_to_book
    
    time_to_book = book_this_room["Time"]

    global room
    
    room = book_this_room["Room"]
    
    print(month_to_book, year_to_book, date_to_book, time_to_book, room)

    import re

    date_to_book = re.sub('\D', '', date_to_book)
    
    global abbrev
    
    abbrev = room[0:3]
    
    abbrev_dict = {"LN-":"process1", "LS-":"process3", "Blo":"process2", "Med":"process2", "Blu":"process3",
                 "Gre":"process3", "Gra":"process3", "Ora":"process3", "UDC":"process3"}
    
    funct = abbrev_dict[abbrev]
    
    if funct == "process1":
        
        ln()
        
    elif funct == "process2":
        
        media_bloom()
        
    elif funct == "process3":
        
        ls_sci_lib_udc()

In [7]:
# books rooms in the fine arts library (LN-____)


def ln():
    
    driver = webdriver.Chrome(service=PATH, options=options)
    
    driver.get("https://www.binghamton.edu/libraries/services/reservations/index.html")
    
    try:
        #-------------------------Bartle Library----------------------------
        #Click Bartle library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN BARTLE"))
        )

        element.click()

        #Click to open drop down menu for bartle library
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']"))
        )

        element.click()


        #Click Show All in the drop down menu
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']/option[5]"))
        )

        element.click()


        today = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if today[0] == "0":
            today = today[1:]
            
        if date_to_book == today:
            
            pass
        
        else:
            
            #Click Go To Date to open Calendar
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
            )

            
            element.click()

            #Click datepicker to select month
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
            )

            element.click()

            #click datepicker to select year
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
            )

            element.click()

            #selecting year given by user in calendar
            element= WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
            element.click()

            #Selecting month given by user in calendar
            months = driver.find_elements(By.TAG_NAME, "span")
            for month in months:
                if month.text == month_to_book:
                    month.click()
                    #time.sleep(5)
                    break

            #selecting date given by user in calender (number date)
            dates = driver.find_elements(By.TAG_NAME, "td")
            
            
            flag = False
            for date in dates:
                if date.text == "1":
                    flag = True

                if flag == False:
                    continue

                if date.text == date_to_book:
                    date.click()
                    time.sleep(5)
                    break

            flag = False
            time.sleep(10)

                    
        tday = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if tday[0] == "0":
            tday = tday[1:]
        
        #scraping available slots for the user given date
        if tday == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

            
        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]


            if "Media Viewing Room" not in details:
                room_ = data[5]
                available_ = data[6] 


            else:
                room_ = data[5] + " " + data[6] + " " + data[7] + " " + data[8]
                available_ = data[9]


            if room_ == room:

                if time_ == time_to_book:

                    slot.click()
                    time.sleep(2)
                    break

        #times = driver.find_element(By.ID, "bookingend_1")
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        
        times = WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/main/div/div/div/div[4]/form/fieldset/div[1]/div/div/div/div/select")))
        
        times.click()
        time.sleep(1)

        end_times = times.find_elements(By.TAG_NAME, "option")
        
        
        n = 1

        for i in end_times:

            print(f"{n}. {i.text}")

            n += 1

        # display list of all possible times you can have the room until:
        
        print("These are all of the available end times.")

        end_time = int(input(f"Which would you like to choose? (1-{str(n-1)}):"))

        end_time2 = end_times[end_time - 1]

        end_time2.click()

        submit = driver.find_element(By.ID, "submit_times")

        submit.click()

        print("Please log into your Binghamton account to finalize your booking.")
        input("Press enter/return when finished.")
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        
        finish = WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.ID, "btn-form-submit")))
        finish.click()

        print("Your room has been booked!")
        print("You will shortly recieve a confirmation email. Happy studying!")
        

    finally:
        driver.quit()
        
        
     
    directory = os.getcwd()

    im = Image.open(f"{directory}/Floor Plans/{room}.jpg")
        
    im.show()
    
        
    print("Would you like to filter again, start a new search, or exit?")
    print("1. Filter Again")
    print("2. New Search")
    print("3. Exit")

    selection = int(input("Your selection: "))

    if selection == 1:

        filter_slots()

    elif selection == 2:

        main()

    else:

        print("Goodbye!")
    
    



In [8]:
# books media viewing room/bloomberg terminal


def media_bloom():

    driver = webdriver.Chrome(service=PATH, options=options)
    
    driver.get("https://www.binghamton.edu/libraries/services/reservations/index.html")
    
    try:
        #-------------------------Bartle Library----------------------------
        #Click Bartle library link
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "RESERVE A STUDY ROOM IN BARTLE"))
        )

        element.click()

        #Click to open drop down menu for bartle library
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']"))
        )

        element.click()


        #Click Show All in the drop down menu
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='gid']/option[5]"))
        )

        element.click()


        today = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if today[0] == "0":
            today = today[1:]

        if date_to_book == today:
            
            pass
        
        else:
            
            #Click Go To Date to open Calendar
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
            )

            
            element.click()

            #Click datepicker to select month
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
            )

            element.click()

            #click datepicker to select year
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
            )

            element.click()

            #selecting year given by user in calendar
            element= WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
            element.click()

            #Selecting month given by user in calendar
            months = driver.find_elements(By.TAG_NAME, "span")
            for month in months:
                if month.text == month_to_book:
                    month.click()
                    #time.sleep(5)
                    break

            #selecting date given by user in calender (number date)
            dates = driver.find_elements(By.TAG_NAME, "td")
            
            flag = False
            for date in dates:
                if date.text == "1":
                    flag = True

                if flag == False:
                    continue

                if date.text == date_to_book:
                    date.click()
                    time.sleep(5)
                    break

            flag = False
            time.sleep(10)
            
                    
        tday = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if tday[0] == "0":
            tday = tday[1:]
        
        #scraping available slots for the user given date
        if tday == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

            
        for slot in available_slots:
            details = slot.get_attribute("title")
            details = details.replace(" - ", " ")
            data = details.split(" ")
            time_ = data[0]
            day_ = data[1].replace(",", "")
            date_ = data[2] + " " + data[3] + " " + data[4]


            if "Media Viewing Room" not in details:
                room_ = data[5]
                available_ = data[6] 


            else:
                room_ = data[5] + " " + data[6] + " " + data[7] + " " + data[8]
                available_ = data[9]


            if room_ == room:

                if time_ == time_to_book:

                    slot.click()
                    time.sleep(1)
                    break

        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        
        times = WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/main/div/div/div/div[4]/form/fieldset/div[1]/div/div/div/div/select")))
        
        times.click()
        time.sleep(1)

        end_times = times.find_elements(By.TAG_NAME, "option")
        
        
        n = 1

        for i in end_times:

            print(f"{n}. {i.text}")

            n += 1

        # display list of all possible times you can have the room until:
        
        print("These are all of the available end times.")

        end_time = int(input(f"Which would you like to choose? (1-{str(n-1)}):"))

        end_time2 = end_times[end_time - 1]

        end_time2.click()

        submit = driver.find_element(By.ID, "submit_times")

        submit.click()

        
        print("Please log into your Binghamton account to finalize your booking.")
        input("Press enter/return when finished.")
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        cont = driver.find_element(By.ID, "terms_accept")
        cont.click()
        time.sleep(3)
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        finish = WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.ID, "btn-form-submit")))
        finish.click()

        print("Your room has been booked!")
        print("You will shortly recieve a confirmation email. Happy studying!")
        


    finally:
        driver.quit()
        
    
    directory = os.getcwd()

    im = Image.open(f"{directory}/Floor Plans/{room}.jpg")
        
    im.show()
    
        
    print("Would you like to filter again, start a new search, or exit?")
    print("1. Filter Again")
    print("2. New Search")
    print("3. Exit")

    selection = int(input("Your selection: "))

    if selection == 1:

        filter_slots()

    elif selection == 2:

        main()

    else:

        print("Goodbye!")

        
        

In [9]:
# books rooms in bartle (LS-____), science library, and UDC


def ls_sci_lib_udc():
    
    num_people = input("How many people are in your group?")
    
    abbrev_dict = {"LS-":"BARTLE", "Blu":"THE SCIENCE LIBRARY", "Gre":"THE SCIENCE LIBRARY", "Gra":"THE SCIENCE LIBRARY", "Ora":"THE SCIENCE LIBRARY", "UDC":"THE UDC"}
    
    loc = abbrev_dict[abbrev]
    
    driver = webdriver.Chrome(service=PATH, options=options)
    
    driver.get("https://www.binghamton.edu/libraries/services/reservations/index.html")
    
    try:
        
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, f"RESERVE A STUDY ROOM IN {loc}"))
        )

        element.click()

        #Click Go To Date to open Calendar
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='eq-time-grid']/div[1]/div[1]/button[1]"))
        )
        
        element.click()
    
        #Click datepicker to select month
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "datepicker-switch"))
        )

        element.click()

        #click datepicker to select year
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//*[@id='equip_']/div[5]/div[2]/table/thead/tr[2]/th[2]"))
        )

        element.click()

         #selecting year given by user in calendar
        element=   WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,f"//span[contains(text(),{year_to_book})]")))
        element.click()

        #Selecting month given by user in calendar
        months = driver.find_elements(By.TAG_NAME, "span")
        for month in months:
            
            if month.text == month_to_book:
                
                month.click()
                break

        #selecting date given by user in calender (number date)
        dates = driver.find_elements(By.TAG_NAME, "td")

        flag = False
        for date in dates:
            if date.text == "1":
                flag = True

            if flag == False:
                continue

            if date.text == date_to_book:
                date.click()
                time.sleep(5)
                break

        flag = False
        time.sleep(10)
        
        tday = datetime.today().strftime('%Y-%m-%d').split("-")[-1]
        if tday[0] == "0":
            tday = tday[1:]
        
        #scraping available slots for the user given date
        if tday == date_to_book:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-today fc-event-future s-lc-eq-avail']")
        else:
            available_slots = driver.find_elements(By.XPATH, "//a[@class='fc-timeline-event fc-h-event fc-event fc-event-start fc-event-end fc-event-future s-lc-eq-avail']")

            
        for slot in available_slots:
            
            if abbrev == "LS-":
                
                details = slot.get_attribute("title")
                details = details.replace(" - ", " ")
                data = details.split(" ")
                time_ = data[0]
                
                if "Media Viewing Room" not in details:
                    room_ = data[5]
                              
                else:
                    room_ = data[5] + " " + data[6] + " " + data[7] + " " + data[8]
                    
                    
            elif abbrev == "UDC":
                
                details = slot.get_attribute("title")
                details = details.replace(" - ", " ")
                data = details.split(" ")
                time_ = data[0]
                
                room_ = data[5] + " " + data[6] + " " + data[7]
                
                
            else:
                
                details = slot.get_attribute("title")
                details = details.replace(" - ", " ")
                data = details.split(" ")
                time_ = data[0]
                
                room_ = data[5] + " " + data[6] + " " + data[7]
                
                
            if room_ == room:

                if time_ == time_to_book:

                    slot.click()
                    break
                    
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        
        times = WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/main/div/div/div/div[4]/form/fieldset/div[1]/div/div/div/div/select")))    
        
        
        times.click()

        end_times = times.find_elements(By.TAG_NAME, "option")
        
        
        n = 1

        for i in end_times:

            print(f"{n}. {i.text}")

            n += 1

        # display list of all possible times you can have the room until:
        
        print("These are all of the available end times.")

        end_time = int(input(f"Which would you like to choose? (1-{str(n-1)}):"))

        end_time2 = end_times[end_time - 1]

        end_time2.click()

        submit = driver.find_element(By.ID, "submit_times")

        submit.click()

        
        print("Please log into your Binghamton account to finalize your booking.")
        input("Press enter/return when finished.")
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        cont = driver.find_element(By.ID, "terms_accept")
        
        cont.click()
        
        people = driver.find_element(By.ID, "q4538")
        people.send_keys(num_people)
        
        
        submit = driver.find_element(By.ID, "btn-form-submit")
        submit.click()
        
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        
        #finish = WebDriverWait(driver,30).until(EC.element_to_be_clickable((By.ID, "btn-form-submit")))
        finish = driver.find_element(By.ID, "btn-form-submit")
        
        finish.click()

        print("Your room has been booked!")
        print("You will shortly recieve a confirmation email. Happy studying!")


    finally:
        driver.quit()
        
        
    directory = os.getcwd()

    im = Image.open(f"{directory}/Floor Plans/{room}.jpg")
        
    im.show()
        
        
    print("Would you like to filter again, start a new search, or exit?")
    print("1. Filter Again")
    print("2. New Search")
    print("3. Exit")

    selection = int(input("Your selection: "))

    if selection == 1:

        filter_slots()

    elif selection == 2:

        main()

    else:

        print("Goodbye!")

        

In [10]:
# main program function:


def main():
    
    show_rooms()
    
    filter_rooms()
    


In [ ]:
main()


Please enter the current year: 2023
Please enter the month in which you'd like to search (Jan-Feb-Mar-Apr-May-Jun-Jul-Aug-Sep-Oct-Nov-Dec): May
Please enter the day you'd like to search as number: 11
       Time       Day          Date               Room Availability
0    9:30pm  Thursday  May 11, 2023           LS-2523A    Available
1   10:00pm  Thursday  May 11, 2023           LS-2523A    Available
2   10:30pm  Thursday  May 11, 2023           LS-2523A    Available
3   11:00pm  Thursday  May 11, 2023           LS-2523A    Available
4   11:30pm  Thursday  May 11, 2023           LS-2523A    Available
..      ...       ...           ...                ...          ...
75  11:30pm  Thursday  May 11, 2023          Bloomberg    Available
76   9:30pm  Thursday  May 11, 2023    Blue Room (108)    Available
77   9:30pm  Thursday  May 11, 2023  Grape Room (G-05)    Available
78   9:30pm  Thursday  May 11, 2023    Grey Room (109)    Available
79   9:30pm  Thursday  May 11, 2023  Orange Room (11